In [104]:
import pandas as pd, numpy as np, re
from pathlib import Path

In [106]:
import pandas as pd, numpy as np

# --- unit helpers (must exist) ---
f_to_c     = lambda f: (f-32.0)*(5/9)
in_to_mm   = lambda x: x*25.4
ft_to_m    = lambda x: x*0.3048
hpa_to_kpa = lambda x: x*0.1

# --- streak helper (missing before) ---
def longest_streak_bool(arr: pd.Series) -> int:
    m = 0; cur = 0
    for v in arr.astype(bool):
        if v: cur += 1; m = max(m, cur)
        else: cur = 0
    return m

# --- daily features ---
GDD_BASE = 10.0  # °C
HEAT_C   = 35.0  # °C

def build_daily(raw: pd.DataFrame) -> pd.DataFrame:
    d = raw.copy()
    d["Year"]  = d["Date"].dt.year
    d["Month"] = d["Date"].dt.month
    # SI units
    d["ppt_mm"]     = in_to_mm(d["ppt (inches)"])
    d["tmin_C"]     = f_to_c(d["tmin (degrees F)"])
    d["tmean_C"]    = f_to_c(d["tmean (degrees F)"])
    d["tmax_C"]     = f_to_c(d["tmax (degrees F)"])
    d["tdmean_C"]   = f_to_c(d["tdmean (degrees F)"])
    d["vpdmin_kPa"] = hpa_to_kpa(d["vpdmin (hPa)"])
    d["vpdmax_kPa"] = hpa_to_kpa(d["vpdmax (hPa)"])
    d["vpdmean_kPa"]= d[["vpdmin_kPa","vpdmax_kPa"]].mean(axis=1)
    d["Elev_m"]     = ft_to_m(d["Elevation (ft)"])
    # thermal
    d["GDD_day"]    = np.maximum(0.0, d["tmean_C"] - GDD_BASE)
    d["HeatDay"]    = (d["tmax_C"] > HEAT_C).astype(int)
    d["FrostDay"]   = (d["tmin_C"] < 0.0).astype(int)
    d["ThermalAmp"] = d["tmax_C"] - d["tmin_C"]
    # water balance + spells
    d["ET0_proxy"]  = d["vpdmean_kPa"] * (d["tmean_C"] + 5).clip(lower=0)  # ET index
    d["P_minus_ET"] = d["ppt_mm"] - d["ET0_proxy"]
    d["DryDay"]     = (d["ppt_mm"] < 1.0).astype(int)      # <1 mm
    d["HeavyRain"]  = (d["ppt_mm"] > 20.0).astype(int)     # >20 mm
    # VPD stress + humidity
    d["VPD_hi_day"] = (d["vpdmax_kPa"] > 2.0).astype(int)
    d["VPD_load"]   = np.maximum(0.0, d["vpdmax_kPa"] - 2.0)
    d["DewDep_C"]   = d["tmean_C"] - d["tdmean_C"]
    return d

# ---- emergence detection ----
def detect_emergence(d: pd.DataFrame) -> pd.DataFrame:
    rows=[]
    for (name, yr), g in d.groupby(["Name","Year"]):
        g = g.sort_values("Date")
        g = g[g["Date"] >= pd.Timestamp(yr,3,1)]
        if g.empty:
            rows.append({"Name":name,"Year":yr,"Emergence":pd.NaT}); continue
        g = g.assign(roll5_Tmean=g["tmean_C"].rolling(5, min_periods=3).mean(),
                     cumGDD=g["GDD_day"].cumsum())
        hit = g[(g["roll5_Tmean"]>10.0) & (g["cumGDD"]>=50.0)]
        emer = hit["Date"].iloc[0] if not hit.empty else pd.NaT
        rows.append({"Name":name,"Year":yr,"Emergence":emer})
    return pd.DataFrame(rows)

# ---- stage tagging by Days After Emergence (DAE) ----
CORN_STAGES = [
    ("VE",   0,   10),
    ("V6",  25,   30),
    ("VT",  55,   59),
    ("R1",  60,   70),
    ("R2",  71,   80),
    ("R3",  81,   90),
    ("R4",  91,  105),
    ("R5", 106,  125),
    ("R6", 126,  160),
]

def tag_corn_stages(d: pd.DataFrame, emer_tbl: pd.DataFrame) -> pd.DataFrame:
    m = d.merge(emer_tbl, on=["Name","Year"], how="left")
    m["DAE"] = (m["Date"] - m["Emergence"]).dt.days
    def stage_from_dae(v):
        if pd.isna(v): return None
        for lab, lo, hi in CORN_STAGES:
            if lo <= v <= hi: return lab
        return None
    m["Stage"] = m["DAE"].apply(stage_from_dae)
    return m

import numpy as np

def corn_stage_aggregate(tagged: pd.DataFrame) -> pd.DataFrame:
    # filter and order
    g = tagged.dropna(subset=["Stage"]).copy()
    g = g.sort_values(["Name","Year","Stage","Date"])

    # per-stage aggregates
    core = (g.groupby(["Name","Year","Stage"], as_index=False)
              .agg(
                  # thermal
                  GDD_sum=("GDD_day","sum"),
                  HeatDays=("HeatDay","sum"),
                  FrostDays=("FrostDay","sum"),
                  Tmean_C=("tmean_C","mean"),
                  Tmax_C=("tmax_C","mean"),
                  ThermalAmp=("ThermalAmp","mean"),
                  # water
                  PPT_mm=("ppt_mm","sum"),
                  ET0_sum=("ET0_proxy","sum"),
                  P_minus_ET=("P_minus_ET","sum"),
                  DryDays=("DryDay","sum"),
                  HeavyRainDays=("HeavyRain","sum"),
                  # VPD / humidity
                  VPDmax_kPa=("vpdmax_kPa","mean"),
                  VPDmean_kPa=("vpdmean_kPa","mean"),
                  VPD_hi_days=("VPD_hi_day","sum"),
                  VPD_load_sum=("VPD_load","sum"),
                  DewDep_C=("DewDep_C","mean"),
              ))

    # longest streaks without the deprecated .apply-on-groups behavior
    grp = g.groupby(["Name","Year","Stage"], observed=True)
    streaks = (pd.DataFrame({
        "LongestDryStreak": grp["DryDay"].apply(longest_streak_bool),
        "LongestHotStreak": grp["HeatDay"].apply(longest_streak_bool),
    }).reset_index())

    out = core.merge(streaks, on=["Name","Year","Stage"], how="left")

    # safe ratio
    out["EffPrecRatio_idx"] = out["PPT_mm"] / out["ET0_sum"].replace(0, np.nan)
    return out


# ---- run ----
# assumes you already defined/load: load_prism_all, DATA_DIR, PATTERN
raw = load_prism_all(DATA_DIR, PATTERN)
daily = build_daily(raw)
emer  = detect_emergence(daily)
corn_tagged = tag_corn_stages(daily, emer)
corn_stage  = corn_stage_aggregate(corn_tagged)

daily.to_parquet("Kashish_results/corn_prism_daily.parquet", index=False)
emer.to_csv("Kashish_results/corn_emergence_dates.csv", index=False)
corn_tagged.to_parquet("Kashish_results/corn_daily_with_stages.parquet", index=False)
corn_stage.to_csv("Kashish_results/corn_stage_features.csv", index=False)

print("Rows daily:", len(daily), "| years:", daily['Year'].min(), "-", daily['Year'].max())
print("Stage rows:", len(corn_stage))


Rows daily: 1534100 | years: 1983 - 2024
Stage rows: 37800


In [50]:
# 1) Wide stage table per Name×Year
import pandas as pd, os
os.makedirs("Kashish_results", exist_ok=True)

corn_stage = pd.read_csv("Kashish_results/corn_stage_features.csv")
wide = (corn_stage.pivot_table(
    index=["Name","Year"], columns="Stage",
    values=["PPT_mm","ET0_sum","P_minus_ET","EffPrecRatio_idx",
            "GDD_sum","HeatDays","FrostDays","LongestDryStreak","LongestHotStreak",
            "Tmean_C","Tmax_C","ThermalAmp",
            "VPDmax_kPa","VPDmean_kPa","VPD_hi_days","VPD_load_sum","DewDep_C"],
    aggfunc="first")
    .reset_index())
wide.columns = ["_".join(c).strip("_") for c in wide.columns.to_flat_index()]
wide.to_csv("Kashish_results/corn_stage_features_wide_plus.csv", index=False)


In [52]:
yld_full = pd.read_csv("Daily_for_transform/crop_yield_1980-2022.csv")
print(yld_full.info())
print(yld_full.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18105 entries, 0 to 18104
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          18105 non-null  int64  
 1   Program             18105 non-null  object 
 2   Year                18105 non-null  int64  
 3   Period              18105 non-null  object 
 4   Week Ending         0 non-null      float64
 5   Geo Level           18105 non-null  object 
 6   State               18105 non-null  object 
 7   State ANSI          18105 non-null  int64  
 8   Ag District         18089 non-null  object 
 9   Ag District Code    18105 non-null  int64  
 10  County              18105 non-null  object 
 11  County ANSI         17359 non-null  float64
 12  Zip Code            0 non-null      float64
 13  Region              0 non-null      float64
 14  watershed_code      18105 non-null  int64  
 15  Watershed           0 non-null      float64
 16  Comm

In [53]:
import pandas as pd

yld = pd.read_csv("crop_yield_1980-2022.csv")

# keep only necessary columns
yld_clean = yld[["Year","County","Commodity","YIELD_LBS_PER_ACRE"]].copy()
yld_clean = yld_clean.rename(columns={"County":"Name", "YIELD_LBS_PER_ACRE":"Yield"})

# check
print(yld_clean.head())
print(yld_clean["Commodity"].unique())


   Year            Name Commodity    Yield
0  2022  OTHER COUNTIES      CORN   6843.2
1  2022  OTHER COUNTIES      CORN  28000.0
2  2022  OTHER COUNTIES    COTTON    871.0
3  2022  OTHER COUNTIES   PEANUTS   4065.0
4  2022  OTHER COUNTIES  SOYBEANS   2214.0
['CORN' 'COTTON' 'PEANUTS' 'SOYBEANS' 'WHEAT']


In [54]:
import pandas as pd
from pathlib import Path

# inputs
yld_path  = "crop_yield_1980-2022.csv"
wide_path = "Kashish_results/corn_stage_features_wide_plus.csv"  # or ..._wide.csv

# 1) clean yields
yld = pd.read_csv(yld_path)
yld_corn = (yld.loc[yld["Commodity"].str.upper()=="CORN", ["Year","County","Commodity","YIELD_LBS_PER_ACRE"]]
               .rename(columns={"County":"Name","YIELD_LBS_PER_ACRE":"Yield"}))

# 2) load stage-wide features
wide = pd.read_csv(wide_path)

# 3) basic sanity
assert {"Name","Year"}.issubset(wide.columns), "Wide table missing Name/Year."

# 4) merge
corn_df = (yld_corn.merge(wide, on=["Name","Year"], how="inner")
                    .dropna()
                    .drop_duplicates(subset=["Name","Year"]))

# 5) save
Path("Kashish_results").mkdir(exist_ok=True, parents=True)
out_path = "Kashish_results/corn_model_table.csv"
corn_df.to_csv(out_path, index=False)

print("rows:", len(corn_df), "years:", corn_df['Year'].min(), "-", corn_df['Year'].max())
print("cols:", len(corn_df.columns))
corn_df.head()


rows: 3327 years: 1983 - 2022
cols: 157


,Year,Name,Commodity,Yield,DewDep_C_R1,DewDep_C_R2,DewDep_C_R3,DewDep_C_R4,DewDep_C_R5,DewDep_C_R6,...,VPDmax_kPa_VT,VPDmean_kPa_R1,VPDmean_kPa_R2,VPDmean_kPa_R3,VPDmean_kPa_R4,VPDmean_kPa_R5,VPDmean_kPa_R6,VPDmean_kPa_V6,VPDmean_kPa_VE,VPDmean_kPa_VT
0,2022,BEAUFORT,CORN,8064.0,5.318182,4.583333,6.977778,6.529630,4.252778,4.544444,...,1.9876,0.802091,0.98625,1.49950,1.294133,1.019650,1.125386,0.923417,0.532318,1.0399
1,2022,CRAVEN,CORN,6501.6,5.323232,5.150000,7.227778,6.796296,4.580556,4.726984,...,2.1604,0.796773,1.06315,1.49965,1.326567,1.080100,1.148000,1.003167,0.551182,1.1756
2,2022,GREENE,CORN,5555.2,5.858586,5.055556,8.050000,7.548148,5.144444,4.777778,...,2.0600,0.883409,1.11840,1.66560,1.495800,1.273425,1.196214,1.035750,0.657545,1.0583
3,2022,HYDE,CORN,8797.6,4.045455,4.222222,6.061111,6.103704,4.144444,4.726984,...,1.5802,0.558545,0.82400,1.18555,1.131633,0.901525,1.098243,0.744500,0.473273,0.8330
4,2022,JOHNSTON,CORN,5448.8,5.464646,7.038889,6.833333,7.659259,4.694444,5.084127,...,1.4062,1.123455,1.43585,1.38205,1.447267,1.142650,1.191943,1.227167,0.853318,0.7384


In [55]:
# soybean_pipeline.py
import pandas as pd
import numpy as np
from pathlib import Path
import os

# --- config ---
DATA_DIR = Path("Daily_for_transform/Daily Climate Data")
PATTERN  = "PRISM_ppt_tmin_tmean_tmax_tdmean_vpdmin_vpdmax_stable_800m_*.csv"
OUTDIR   = Path("Kashish_results"); OUTDIR.mkdir(parents=True, exist_ok=True)

# ---------- soybean phenology (DAE windows) ----------
SOYBEAN_STAGES = [
    ("VE",    0,   7),
    ("V2_V3", 8,  25),
    ("V4_V6", 26, 45),
    ("R1_R2", 46, 65),   # flowering
    ("R3_R4", 66, 85),   # pod set
    ("R5_R6", 86, 115),  # seed fill
    ("R7_R8", 116, 140), # maturity
]

def tag_soybean_stages(d: pd.DataFrame, emer_tbl: pd.DataFrame) -> pd.DataFrame:
    m = d.merge(emer_tbl, on=["Name","Year"], how="left")
    m["DAE"] = (m["Date"] - m["Emergence"]).dt.days
    def stage_from_dae(v):
        if pd.isna(v): return None
        for lab, lo, hi in SOYBEAN_STAGES:
            if lo <= v <= hi: return lab
        return None
    m["Stage"] = m["DAE"].apply(stage_from_dae)
    return m

def soybean_stage_aggregate(tagged: pd.DataFrame) -> pd.DataFrame:
    g = tagged.dropna(subset=["Stage"]).copy()
    g = g.sort_values(["Name","Year","Stage","Date"])
    core = (g.groupby(["Name","Year","Stage"], as_index=False)
              .agg(
                  # thermal
                  GDD_sum=("GDD_day","sum"),
                  HeatDays=("HeatDay","sum"),
                  FrostDays=("FrostDay","sum"),
                  Tmean_C=("tmean_C","mean"),
                  Tmax_C=("tmax_C","mean"),
                  ThermalAmp=("ThermalAmp","mean"),
                  # water
                  PPT_mm=("ppt_mm","sum"),
                  ET0_sum=("ET0_proxy","sum"),
                  P_minus_ET=("P_minus_ET","sum"),
                  DryDays=("DryDay","sum"),
                  HeavyRainDays=("HeavyRain","sum"),
                  # VPD / humidity
                  VPDmax_kPa=("vpdmax_kPa","mean"),
                  VPDmean_kPa=("vpdmean_kPa","mean"),
                  VPD_hi_days=("VPD_hi_day","sum"),
                  VPD_load_sum=("VPD_load","sum"),
                  DewDep_C=("DewDep_C","mean"),
              ))
    # longest streaks (uses your longest_streak_bool)
    grp = g.groupby(["Name","Year","Stage"], observed=True)
    streaks = (pd.DataFrame({
        "LongestDryStreak": grp["DryDay"].apply(longest_streak_bool),
        "LongestHotStreak": grp["HeatDay"].apply(longest_streak_bool),
    }).reset_index())
    out = core.merge(streaks, on=["Name","Year","Stage"], how="left")
    out["EffPrecRatio_idx"] = out["PPT_mm"] / out["ET0_sum"].replace(0, np.nan)
    return out

# ---------- run ----------
# reuse your robust loader and daily builder
raw   = load_prism_all(DATA_DIR, PATTERN)   # already tested for corn
daily = build_daily(raw)                    # already extended with ET/VPD features
emer  = detect_emergence(daily)             # VE per Name×Year

# stage tagging and aggregation
soy_tagged = tag_soybean_stages(daily, emer)
soy_stage  = soybean_stage_aggregate(soy_tagged)

# save long and wide
soy_tagged.to_parquet(OUTDIR/"soybean_daily_with_stages.parquet", index=False)
soy_stage.to_csv(OUTDIR/"soybean_stage_features.csv", index=False)

wide = (soy_stage.pivot_table(
    index=["Name","Year"], columns="Stage",
    values=["PPT_mm","ET0_sum","P_minus_ET","EffPrecRatio_idx",
            "GDD_sum","HeatDays","FrostDays","LongestDryStreak","LongestHotStreak",
            "Tmean_C","Tmax_C","ThermalAmp",
            "VPDmax_kPa","VPDmean_kPa","VPD_hi_days","VPD_load_sum","DewDep_C"],
    aggfunc="first").reset_index())
wide.columns = ["_".join(c).strip("_") for c in wide.columns.to_flat_index()]
wide.to_csv(OUTDIR/"soybean_stage_features_wide_plus.csv", index=False)

print("Soybean stage rows:", len(soy_stage), "| wide rows:", wide.shape)


Soybean stage rows: 29400 | wide rows: (4200, 121)


In [56]:
# merge soybean yields
yld = pd.read_csv("crop_yield_1980-2022.csv")
yld_soy = (yld.loc[yld["Commodity"].str.upper()=="SOYBEANS", ["Year","County","Commodity","YIELD_LBS_PER_ACRE"]]
              .rename(columns={"County":"Name","YIELD_LBS_PER_ACRE":"Yield"}))

soy_df = (yld_soy.merge(wide[wide["Year"]<=2022], on=["Name","Year"], how="inner")
                 .dropna()
                 .drop_duplicates(subset=["Name","Year"]))
soy_df.to_csv(OUTDIR/"soybean_model_table.csv", index=False)

# inference table for 2023
soy_infer_2023 = (wide[wide["Year"]==2023]
                  .dropna()
                  .drop_duplicates(subset=["Name","Year"]))
soy_infer_2023.to_csv(OUTDIR/"soybean_inference_2023.csv", index=False)

print("Model table:", soy_df.shape, "| 2023 inference:", soy_infer_2023.shape)


Model table: (3096, 123) | 2023 inference: (100, 121)


In [57]:
# peanut_pipeline.py
import pandas as pd, numpy as np, re, os
from pathlib import Path

# =================== CONFIG ===================
DATA_DIR = Path("Daily_for_transform/Daily Climate Data")
PATTERN  = "PRISM_ppt_tmin_tmean_tmax_tdmean_vpdmin_vpdmax_stable_800m_*.csv"
OUTDIR   = Path("Kashish_results/peanuts"); OUTDIR.mkdir(parents=True, exist_ok=True)
YIELD_CSV = "crop_yield_1980-2022.csv"

# =================== LOADER (robust) ===================
EXP = ["Name","Longitude","Latitude","Elevation (ft)","Date",
       "ppt (inches)","tmin (degrees F)","tmean (degrees F)","tmax (degrees F)",
       "tdmean (degrees F)","vpdmin (hPa)","vpdmax (hPa)"]

def _clean_num(s):
    if pd.isna(s): return np.nan
    s = re.sub(r'(?<=\d)\.(?=\s*$)', '', str(s).strip())
    s = re.sub(r'[^0-9.\-eE]', '', s)
    try: return float(s)
    except: return np.nan

def _find_header_idx(path: Path, max_scan=80):
    with path.open("r", encoding="utf-8-sig", errors="ignore") as f:
        for i in range(max_scan):
            line = f.readline()
            if not line: break
            low = line.strip().lower()
            if low.startswith("name,longitude,latitude,elevation (ft),date,ppt"): return i
            if "name," in low and ",date," in low: return i
    return None

def read_one_prism(path: Path) -> pd.DataFrame:
    hdr = _find_header_idx(path)
    if hdr is None: hdr = 9
    df = pd.read_csv(path, header=hdr, engine="python", on_bad_lines="skip")
    if "Date" in df.columns:
        df = df[df["Date"].astype(str).str.match(r"\d{4}-\d{2}-\d{2}", na=False)]
    miss = [c for c in EXP if c not in df.columns]
    if miss: raise ValueError(f"{path.name}: missing {miss}")
    for c in EXP:
        if c not in ["Name","Date"]:
            df[c] = df[c].apply(_clean_num)
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"]).copy()
    df["__src"] = path.name
    return df

def load_prism_all(root: Path, pattern: str) -> pd.DataFrame:
    files = sorted(root.rglob(pattern))
    if not files: raise FileNotFoundError(f"No files under {root.resolve()}")
    parts, errs = [], []
    for p in files:
        try: parts.append(read_one_prism(p))
        except Exception as e: errs.append((p.name, str(e)))
    if not parts: raise RuntimeError("Parsed 0 valid tables.")
    out = pd.concat(parts, ignore_index=True)
    out = out.drop_duplicates(subset=["Name","Date","ppt (inches)","tmin (degrees F)",
                                      "tmean (degrees F)","tmax (degrees F)","tdmean (degrees F)",
                                      "vpdmin (hPa)","vpdmax (hPa)"])
    return out

# =================== DAILY FEATURES ===================
f_to_c     = lambda f: (f-32.0)*(5/9)
in_to_mm   = lambda x: x*25.4
ft_to_m    = lambda x: x*0.3048
hpa_to_kpa = lambda x: x*0.1

HEAT_C   = 35.0  # generic heat-stress threshold

def build_daily(raw: pd.DataFrame) -> pd.DataFrame:
    d = raw.copy()
    d["Year"]  = d["Date"].dt.year
    d["Month"] = d["Date"].dt.month
    # SI and derivatives
    d["ppt_mm"]     = in_to_mm(d["ppt (inches)"])
    d["tmin_C"]     = f_to_c(d["tmin (degrees F)"])
    d["tmean_C"]    = f_to_c(d["tmean (degrees F)"])
    d["tmax_C"]     = f_to_c(d["tmax (degrees F)"])
    d["tdmean_C"]   = f_to_c(d["tdmean (degrees F)"])
    d["vpdmin_kPa"] = hpa_to_kpa(d["vpdmin (hPa)"])
    d["vpdmax_kPa"] = hpa_to_kpa(d["vpdmax (hPa)"])
    d["vpdmean_kPa"]= d[["vpdmin_kPa","vpdmax_kPa"]].mean(axis=1)
    d["Elev_m"]     = ft_to_m(d["Elevation (ft)"])
    # generic thermal
    d["HeatDay"]    = (d["tmax_C"] > HEAT_C).astype(int)
    d["FrostDay"]   = (d["tmin_C"] < 0.0).astype(int)
    d["ThermalAmp"] = d["tmax_C"] - d["tmin_C"]
    # water + VPD
    d["ET0_proxy"]  = d["vpdmean_kPa"] * (d["tmean_C"] + 5).clip(lower=0)
    d["P_minus_ET"] = d["ppt_mm"] - d["ET0_proxy"]
    d["DryDay"]     = (d["ppt_mm"] < 1.0).astype(int)
    d["HeavyRain"]  = (d["ppt_mm"] > 20.0).astype(int)
    d["VPD_hi_day"] = (d["vpdmax_kPa"] > 2.0).astype(int)
    d["VPD_load"]   = np.maximum(0.0, d["vpdmax_kPa"] - 2.0)
    d["DewDep_C"]   = d["tmean_C"] - d["tdmean_C"]
    return d

# =================== EMERGENCE (PEANUT-SPECIFIC) ===================
# Use base 12.8 °C for detection.
def detect_emergence_peanut(d: pd.DataFrame) -> pd.DataFrame:
    rows=[]
    for (name, yr), g in d.groupby(["Name","Year"]):
        g = g.sort_values("Date")
        g = g[g["Date"] >= pd.Timestamp(yr,3,1)]
        if g.empty: rows.append({"Name":name,"Year":yr,"Emergence":pd.NaT}); continue
        g["GDD12"] = np.maximum(0.0, g["tmean_C"] - 12.8)
        g["roll5_Tmean"] = g["tmean_C"].rolling(5, min_periods=3).mean()
        g["cumGDD12"]    = g["GDD12"].cumsum()
        hit = g[(g["roll5_Tmean"]>12.0) & (g["cumGDD12"]>=60.0)]
        emer = hit["Date"].iloc[0] if not hit.empty else pd.NaT
        rows.append({"Name":name,"Year":yr,"Emergence":emer})
    return pd.DataFrame(rows)

# =================== STAGES (PEANUT) ===================
PEANUT_STAGES = [
    ("VE",        0,   10),
    ("VEG",      11,   30),
    ("PEG",      31,   60),
    ("POD_SET",  61,   90),
    ("SEED_FILL",91,  120),
    ("MATURITY",121,  150),
]

def tag_peanut_stages(d: pd.DataFrame, emer_tbl: pd.DataFrame) -> pd.DataFrame:
    m = d.merge(emer_tbl, on=["Name","Year"], how="left")
    m["DAE"] = (m["Date"] - m["Emergence"]).dt.days
    # peanut-specific GDD base 12.8 °C for aggregation
    m["GDD_day"] = np.maximum(0.0, m["tmean_C"] - 12.8)
    def stage_from_dae(v):
        if pd.isna(v): return None
        for lab, lo, hi in PEANUT_STAGES:
            if lo <= v <= hi: return lab
        return None
    m["Stage"] = m["DAE"].apply(stage_from_dae)
    return m

# =================== STREAKS + AGG ===================
def longest_streak_bool(arr: pd.Series) -> int:
    m = 0; cur = 0
    for v in arr.astype(bool):
        if v: cur += 1; m = max(m, cur)
        else: cur = 0
    return m

def peanut_stage_aggregate(tagged: pd.DataFrame) -> pd.DataFrame:
    g = tagged.dropna(subset=["Stage"]).copy().sort_values(["Name","Year","Stage","Date"])
    core = (g.groupby(["Name","Year","Stage"], as_index=False)
              .agg(
                  GDD_sum=("GDD_day","sum"),
                  HeatDays=("HeatDay","sum"),
                  FrostDays=("FrostDay","sum"),
                  Tmean_C=("tmean_C","mean"),
                  Tmax_C=("tmax_C","mean"),
                  ThermalAmp=("ThermalAmp","mean"),
                  PPT_mm=("ppt_mm","sum"),
                  ET0_sum=("ET0_proxy","sum"),
                  P_minus_ET=("P_minus_ET","sum"),
                  DryDays=("DryDay","sum"),
                  HeavyRainDays=("HeavyRain","sum"),
                  VPDmax_kPa=("vpdmax_kPa","mean"),
                  VPDmean_kPa=("vpdmean_kPa","mean"),
                  VPD_hi_days=("VPD_hi_day","sum"),
                  VPD_load_sum=("VPD_load","sum"),
                  DewDep_C=("DewDep_C","mean"),
              ))
    grp = g.groupby(["Name","Year","Stage"], observed=True)
    streaks = (pd.DataFrame({
        "LongestDryStreak": grp["DryDay"].apply(longest_streak_bool),
        "LongestHotStreak": grp["HeatDay"].apply(longest_streak_bool),
    }).reset_index())
    out = core.merge(streaks, on=["Name","Year","Stage"], how="left")
    out["EffPrecRatio_idx"] = out["PPT_mm"] / out["ET0_sum"].replace(0, np.nan)
    return out

# =================== RUN ===================
raw   = load_prism_all(DATA_DIR, PATTERN)
daily = build_daily(raw)
emer  = detect_emergence_peanut(daily)
pea_tagged = tag_peanut_stages(daily, emer)
pea_stage  = peanut_stage_aggregate(pea_tagged)

# save
pea_tagged.to_parquet(OUTDIR/"peanut_daily_with_stages.parquet", index=False)
pea_stage.to_csv(OUTDIR/"peanut_stage_features.csv", index=False)

# wide
wide = (pea_stage.pivot_table(
    index=["Name","Year"], columns="Stage",
    values=["PPT_mm","ET0_sum","P_minus_ET","EffPrecRatio_idx",
            "GDD_sum","HeatDays","FrostDays","LongestDryStreak","LongestHotStreak",
            "Tmean_C","Tmax_C","ThermalAmp",
            "VPDmax_kPa","VPDmean_kPa","VPD_hi_days","VPD_load_sum","DewDep_C"],
    aggfunc="first").reset_index())
wide.columns = ["_".join(c).strip("_") for c in wide.columns.to_flat_index()]
wide.to_csv(OUTDIR/"peanut_stage_features_wide_plus.csv", index=False)

print("Peanut stage rows:", len(pea_stage), "| wide rows:", wide.shape)

# =================== MERGE YIELDS ≤2022 + INFERENCE 2023 ===================
yld = pd.read_csv(YIELD_CSV)
yld_pea = (yld.loc[yld["Commodity"].str.upper()=="PEANUTS", ["Year","County","Commodity","YIELD_LBS_PER_ACRE"]]
             .rename(columns={"County":"Name","YIELD_LBS_PER_ACRE":"Yield"}))

pea_df = (yld_pea.merge(wide[wide["Year"]<=2022], on=["Name","Year"], how="inner")
                 .dropna()
                 .drop_duplicates(subset=["Name","Year"]))
pea_df.to_csv(OUTDIR/"peanut_model_table.csv", index=False)

pea_infer_2023 = (wide[wide["Year"]==2023]
                  .dropna()
                  .drop_duplicates(subset=["Name","Year"]))
pea_infer_2023.to_csv(OUTDIR/"peanut_inference_2023.csv", index=False)

print("Model table:", pea_df.shape, "| 2023 inference:", pea_infer_2023.shape)



Peanut stage rows: 25200 | wide rows: (4200, 104)
Model table: (938, 106) | 2023 inference: (100, 104)
Saved: Kashish_results/peanuts/peanut_predictions_2023.csv


In [ ]:
# =================== BASELINE MODEL → 2023 PREDICTIONS ===================
FEATS = [c for c in pea_df.columns if c not in ["Name","Year","Commodity","Yield"]]

# XGBoost
try:
    from xgboost import XGBRegressor
    model = XGBRegressor(
        n_estimators=600, max_depth=6, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0, random_state=42
    )
except Exception:
    from sklearn.ensemble import RandomForestRegressor
    model = RandomForestRegressor(n_estimators=500, max_depth=None, random_state=42, n_jobs=-1)

model.fit(pea_df[FEATS], pea_df["Yield"])
pred23 = model.predict(pea_infer_2023[FEATS])

predictions_peanut = pd.DataFrame({
    "Year": 2023,
    "Commodity": "PEANUTS",
    "Name": pea_infer_2023["Name"].values,
    "Value": pred23
})
predictions_peanut.to_csv(OUTDIR/"peanut_predictions_2023.csv", index=False)
print("Saved:", OUTDIR/"peanut_predictions_2023.csv")


In [59]:
# ==== BOOTSTRAP HELPERS (run once if not already defined) ====
import pandas as pd, numpy as np, re
from pathlib import Path

# --- robust PRISM loader (header auto-detect) ---
EXP = ["Name","Longitude","Latitude","Elevation (ft)","Date",
       "ppt (inches)","tmin (degrees F)","tmean (degrees F)","tmax (degrees F)",
       "tdmean (degrees F)","vpdmin (hPa)","vpdmax (hPa)"]

def _clean_num(s):
    if pd.isna(s): return np.nan
    s = re.sub(r'(?<=\d)\.(?=\s*$)', '', str(s).strip())
    s = re.sub(r'[^0-9.\-eE]', '', s)
    try: return float(s)
    except: return np.nan

def _find_header_idx(path: Path, max_scan=80):
    with path.open("r", encoding="utf-8-sig", errors="ignore") as f:
        for i in range(max_scan):
            line = f.readline()
            if not line: break
            low = line.strip().lower()
            if low.startswith("name,longitude,latitude,elevation (ft),date,ppt"): return i
            if "name," in low and ",date," in low: return i
    return None

def read_one_prism(path: Path) -> pd.DataFrame:
    hdr = _find_header_idx(path);  hdr = 9 if hdr is None else hdr
    df = pd.read_csv(path, header=hdr, engine="python", on_bad_lines="skip")
    if "Date" in df.columns:
        df = df[df["Date"].astype(str).str.match(r"\d{4}-\d{2}-\d{2}", na=False)]
    miss = [c for c in EXP if c not in df.columns]
    if miss: raise ValueError(f"{path.name}: missing {miss}")
    for c in EXP:
        if c not in ["Name","Date"]:
            df[c] = df[c].apply(_clean_num)
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"]).copy()
    df["__src"] = path.name
    return df

def load_prism_all(root: Path, pattern: str) -> pd.DataFrame:
    files = sorted(root.rglob(pattern))
    if not files: raise FileNotFoundError(f"No files under {root.resolve()}")
    parts = [read_one_prism(p) for p in files]
    out = pd.concat(parts, ignore_index=True)
    out = out.drop_duplicates(subset=["Name","Date","ppt (inches)","tmin (degrees F)",
                                      "tmean (degrees F)","tmax (degrees F)","tdmean (degrees F)",
                                      "vpdmin (hPa)","vpdmax (hPa)"])
    return out

# --- unit helpers + daily feature engineer (ET/VPD/heat/dry etc.) ---
f_to_c     = lambda f: (f-32.0)*(5/9)
in_to_mm   = lambda x: x*25.4
ft_to_m    = lambda x: x*0.3048
hpa_to_kpa = lambda x: x*0.1
HEAT_C     = 35.0

def build_daily(raw: pd.DataFrame) -> pd.DataFrame:
    d = raw.copy()
    d["Year"]  = d["Date"].dt.year
    d["Month"] = d["Date"].dt.month
    d["ppt_mm"]     = in_to_mm(d["ppt (inches)"])
    d["tmin_C"]     = f_to_c(d["tmin (degrees F)"])
    d["tmean_C"]    = f_to_c(d["tmean (degrees F)"])
    d["tmax_C"]     = f_to_c(d["tmax (degrees F)"])
    d["tdmean_C"]   = f_to_c(d["tdmean (degrees F)"])
    d["vpdmin_kPa"] = hpa_to_kpa(d["vpdmin (hPa)"])
    d["vpdmax_kPa"] = hpa_to_kpa(d["vpdmax (hPa)"])
    d["vpdmean_kPa"]= d[["vpdmin_kPa","vpdmax_kPa"]].mean(axis=1)
    d["Elev_m"]     = ft_to_m(d["Elevation (ft)"])
    d["HeatDay"]    = (d["tmax_C"] > HEAT_C).astype(int)
    d["FrostDay"]   = (d["tmin_C"] < 0.0).astype(int)
    d["ThermalAmp"] = d["tmax_C"] - d["tmin_C"]
    d["ET0_proxy"]  = d["vpdmean_kPa"] * (d["tmean_C"] + 5).clip(lower=0)
    d["P_minus_ET"] = d["ppt_mm"] - d["ET0_proxy"]
    d["DryDay"]     = (d["ppt_mm"] < 1.0).astype(int)
    d["HeavyRain"]  = (d["ppt_mm"] > 20.0).astype(int)
    d["VPD_hi_day"] = (d["vpdmax_kPa"] > 2.0).astype(int)
    d["VPD_load"]   = np.maximum(0.0, d["vpdmax_kPa"] - 2.0)
    d["DewDep_C"]   = d["tmean_C"] - d["tdmean_C"]
    return d

# --- streak helper ---
def longest_streak_bool(arr: pd.Series) -> int:
    m = 0; cur = 0
    for v in arr.astype(bool):
        if v: cur += 1; m = max(m, cur)
        else: cur = 0
    return m


In [61]:
# ------------wheat_pipeline.py------
import pandas as pd, numpy as np
from pathlib import Path

# -------- CONFIG --------
DATA_DIR = Path("Daily_for_transform/Daily Climate Data")
PATTERN  = "PRISM_ppt_tmin_tmean_tmax_tdmean_vpdmin_vpdmax_stable_800m_*.csv"
OUTDIR   = Path("Kashish_results"); OUTDIR.mkdir(parents=True, exist_ok=True)
YIELD_CSV = "crop_yield_1980-2022.csv"

# -------- Wheat specifics --------
# GDD base commonly ~4.4 °C (40°F). Heat stress threshold milder than corn; we already mark HeatDay at 35°C.
WHEAT_GDD_BASE = 4.4

# Stage windows in Days After Emergence (DAE), non-overlapping for features
WHEAT_STAGES = [
    ("TILLER",       0,   60),
    ("STEM_ELONG",  61,  110),
    ("BOOT",       111,  135),
    ("HEAD_FLOWER",136,  155),
    ("GRAIN_FILL", 156,  190),
    ("RIPEN",      191,  220),
]

def detect_wheat_emergence_seasons(daily: pd.DataFrame) -> pd.DataFrame:
    """
    Build a table of wheat seasons per county with Emergence possibly in the previous fall.
    SeasonYear = harvest year. Try Sep 1–Dec 31 of Year-1; fallback Jan 1–Apr 1 of Year.
    Criteria: 5-day mean Tmean > 4 °C AND cumulative GDD_base4.4 ≥ 80 within the window.
    """
    out = []
    names = daily["Name"].unique()
    yr_min, yr_max = daily["Date"].dt.year.min(), daily["Date"].dt.year.max()
    for name in names:
        dn = daily[daily["Name"]==name].sort_values("Date").copy()
        for season_year in range(max(yr_min+1, 1983), yr_max+1):
            # 1) primary window: previous fall
            fall0 = pd.Timestamp(season_year-1, 9, 1)
            fall1 = pd.Timestamp(season_year-1, 12, 31)
            g = dn[(dn["Date"]>=fall0) & (dn["Date"]<=fall1)].copy()
            g["GDD4"] = np.maximum(0.0, g["tmean_C"] - WHEAT_GDD_BASE)
            g["roll5_Tmean"] = g["tmean_C"].rolling(5, min_periods=3).mean()
            g["cumGDD4"] = g["GDD4"].cumsum()
            hit = g[(g["roll5_Tmean"]>4.0) & (g["cumGDD4"]>=80.0)]
            if not hit.empty:
                emer = hit["Date"].iloc[0]
                out.append({"Name":name, "SeasonYear":season_year, "Emergence":emer})
                continue
            # 2) fallback: early spring of season year
            spr0 = pd.Timestamp(season_year, 1, 1)
            spr1 = pd.Timestamp(season_year, 4, 1)
            g2 = dn[(dn["Date"]>=spr0) & (dn["Date"]<=spr1)].copy()
            if g2.empty:
                continue
            g2["GDD4"] = np.maximum(0.0, g2["tmean_C"] - WHEAT_GDD_BASE)
            g2["roll5_Tmean"] = g2["tmean_C"].rolling(5, min_periods=3).mean()
            g2["cumGDD4"] = g2["GDD4"].cumsum()
            hit2 = g2[(g2["roll5_Tmean"]>4.0) & (g2["cumGDD4"]>=80.0)]
            emer2 = hit2["Date"].iloc[0] if not hit2.empty else pd.NaT
            out.append({"Name":name, "SeasonYear":season_year, "Emergence":emer2})
    return pd.DataFrame(out)

def tag_wheat_stages(daily: pd.DataFrame, seasons: pd.DataFrame) -> pd.DataFrame:
    """
    Assign SeasonYear and Stage by iterating season rows and slicing the daily table.
    DAE computed against Emergence; rows outside [Emergence, Emergence+220] are ignored for wheat.
    Adds wheat-specific GDD_day (base 4.4 °C) for stage aggregation.
    """
    parts = []
    for r in seasons.itertuples(index=False):
        if pd.isna(r.Emergence): 
            continue
        lo = r.Emergence
        hi = r.Emergence + pd.Timedelta(days=220)
        slab = daily[(daily["Name"]==r.Name) & (daily["Date"]>=lo) & (daily["Date"]<=hi)].copy()
        if slab.empty:
            continue
        slab["SeasonYear"] = r.SeasonYear
        slab["DAE"] = (slab["Date"] - r.Emergence).dt.days
        # wheat-specific GDD
        slab["GDD_day"] = np.maximum(0.0, slab["tmean_C"] - WHEAT_GDD_BASE)
        # stage label
        def stage_from_dae(v):
            for lab, lo_d, hi_d in WHEAT_STAGES:
                if lo_d <= v <= hi_d: return lab
            return None
        slab["Stage"] = slab["DAE"].apply(stage_from_dae)
        parts.append(slab)
    if not parts:
        return pd.DataFrame()
    return pd.concat(parts, ignore_index=True)

def wheat_stage_aggregate(tagged: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregates the same rich feature set you used for other crops, but by SeasonYear.
    """
    g = tagged.dropna(subset=["Stage"]).copy().sort_values(["Name","SeasonYear","Stage","Date"])
    # core aggregates
    core = (g.groupby(["Name","SeasonYear","Stage"], as_index=False)
              .agg(
                  # thermal
                  GDD_sum=("GDD_day","sum"),
                  HeatDays=("HeatDay","sum"),
                  FrostDays=("FrostDay","sum"),
                  Tmean_C=("tmean_C","mean"),
                  Tmax_C=("tmax_C","mean"),
                  ThermalAmp=("ThermalAmp","mean"),
                  # water
                  PPT_mm=("ppt_mm","sum"),
                  ET0_sum=("ET0_proxy","sum"),
                  P_minus_ET=("P_minus_ET","sum"),
                  DryDays=("DryDay","sum"),
                  HeavyRainDays=("HeavyRain","sum"),
                  # VPD / humidity
                  VPDmax_kPa=("vpdmax_kPa","mean"),
                  VPDmean_kPa=("vpdmean_kPa","mean"),
                  VPD_hi_days=("VPD_hi_day","sum"),
                  VPD_load_sum=("VPD_load","sum"),
                  DewDep_C=("DewDep_C","mean"),
              ))
    # streaks
    grp = g.groupby(["Name","SeasonYear","Stage"], observed=True)
    streaks = (pd.DataFrame({
        "LongestDryStreak": grp["DryDay"].apply(longest_streak_bool),
        "LongestHotStreak": grp["HeatDay"].apply(longest_streak_bool),
    }).reset_index())
    out = core.merge(streaks, on=["Name","SeasonYear","Stage"], how="left")
    out["EffPrecRatio_idx"] = out["PPT_mm"] / out["ET0_sum"].replace(0, np.nan)
    return out

# ================= RUN =================
# 1) Load and daily features (reuse your robust loader and extended daily builder)
raw   = load_prism_all(DATA_DIR, PATTERN)
daily = build_daily(raw)  # must include ET0_proxy, P_minus_ET, HeatDay, DryDay, VPD*, etc.

# 2) Detect wheat seasons and emergence (cross-year)
wht_seasons = detect_wheat_emergence_seasons(daily)
wht_tagged  = tag_wheat_stages(daily, wht_seasons)

# 3) Aggregate by SeasonYear×Stage
wht_stage = wheat_stage_aggregate(wht_tagged)

# 4) Save long and wide
wht_tagged.to_parquet(OUTDIR/"wheat_daily_with_stages.parquet", index=False)
wht_stage.to_csv(OUTDIR/"wheat_stage_features.csv", index=False)

# 5) Pivot to wide per Name×SeasonYear; rename SeasonYear -> Year to match yields
wide = (wht_stage.pivot_table(
    index=["Name","SeasonYear"], columns="Stage",
    values=["PPT_mm","ET0_sum","P_minus_ET","EffPrecRatio_idx",
            "GDD_sum","HeatDays","FrostDays","LongestDryStreak","LongestHotStreak",
            "Tmean_C","Tmax_C","ThermalAmp",
            "VPDmax_kPa","VPDmean_kPa","VPD_hi_days","VPD_load_sum","DewDep_C"],
    aggfunc="first").reset_index())
wide.columns = ["_".join(c).strip("_") for c in wide.columns.to_flat_index()]
wide = wide.rename(columns={"SeasonYear":"Year"})
wide.to_csv(OUTDIR/"wheat_stage_features_wide_plus.csv", index=False)

print("Wheat stage rows:", len(wht_stage), "| wide rows:", wide.shape)




Wheat stage rows: 24600 | wide rows: (4100, 104)


In [63]:
# 6) Merge yields ≤2022 and prepare 2023 inference
yld = pd.read_csv(YIELD_CSV)
# Use any wheat entry: includes "WHEAT" or "WHEAT, WINTER" etc.
is_wheat = yld["Commodity"].str.upper().str.contains("WHEAT")
yld_wht = (yld.loc[is_wheat, ["Year","County","Commodity","YIELD_LBS_PER_ACRE"]]
              .rename(columns={"County":"Name","YIELD_LBS_PER_ACRE":"Yield"}))

wht_df = (yld_wht.merge(wide[wide["Year"]<=2022], on=["Name","Year"], how="inner")
                   .dropna()
                   .drop_duplicates(subset=["Name","Year"]))
wht_df.to_csv(OUTDIR/"wheat_model_table.csv", index=False)



In [64]:
wht_infer_2023 = (wide[wide["Year"]==2023]
                  .dropna()
                  .drop_duplicates(subset=["Name","Year"]))
wht_infer_2023.to_csv(OUTDIR/"wheat_inference_2023.csv", index=False)

print("Model table:", wht_df.shape, "| 2023 inference:", wht_infer_2023.shape)

# 7) Baseline model → 2023 predictions
FEATS = [c for c in wht_df.columns if c not in ["Name","Year","Commodity","Yield"]]

try:
    from xgboost import XGBRegressor
    model = XGBRegressor(
        n_estimators=600, max_depth=6, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0, random_state=42
    )
except Exception:
    from sklearn.ensemble import RandomForestRegressor
    model = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)

model.fit(wht_df[FEATS], wht_df["Yield"])
pred23 = model.predict(wht_infer_2023[FEATS])

predictions_wheat = pd.DataFrame({
    "Year": 2023,
    "Commodity": "WHEAT",
    "Name": wht_infer_2023["Name"].values,
    "Value": pred23
})
predictions_wheat.to_csv(OUTDIR/"wheat_predictions_2023.csv", index=False)
print("Saved:", OUTDIR/"wheat_predictions_2023.csv")

Model table: (2836, 106) | 2023 inference: (100, 104)
Saved: Kashish_results/wheat_predictions_2023.csv


In [65]:
# cotton_pipeline.py
import pandas as pd, numpy as np
from pathlib import Path

# ===== CONFIG =====
DATA_DIR = Path("/home/ec2-user/SageMaker/PSI Hackathon/Daily_for_transform/Daily Climate Data")
PATTERN  = "PRISM_ppt_tmin_tmean_tmax_tdmean_vpdmin_vpdmax_stable_800m_*.csv"
OUTDIR   = Path("Kashish_results"); OUTDIR.mkdir(parents=True, exist_ok=True)
YIELD_CSV = "crop_yield_1980-2022.csv"

# ===== COTTON PHENOLOGY =====
# Base temp ~15.6°C (60°F). Use for emergence + GDD.
COTTON_GDD_BASE = 15.6

# Non-overlapping DAE windows for features (NC-typical lengths)
COTTON_STAGES = [
    ("VE",          0,   10),   # emergence
    ("VEG",        11,   35),   # vegetative expansion
    ("SQUARING",   36,   55),   # first squares
    ("FLOWERING",  56,   85),   # first bloom to peak bloom
    ("BOLL_DEV",   86,  120),   # boll development/fill
    ("OPEN_MAT",  121,  160),   # open boll to cutout/maturity
]

# ===== Emergence detection (cotton-specific) =====
def detect_emergence_cotton(d: pd.DataFrame) -> pd.DataFrame:
    """Per Name×Year: first date ≥ Mar 1 where roll-5 Tmean > 15°C and cum GDD15.6 ≥ 60."""
    rows=[]
    for (name, yr), g in d.groupby(["Name","Year"]):
        g = g.sort_values("Date")
        g = g[g["Date"] >= pd.Timestamp(yr,3,1)].copy()
        if g.empty:
            rows.append({"Name":name,"Year":yr,"Emergence":pd.NaT}); continue
        g["GDD15"] = np.maximum(0.0, g["tmean_C"] - COTTON_GDD_BASE)
        g["roll5_Tmean"] = g["tmean_C"].rolling(5, min_periods=3).mean()
        g["cumGDD15"]    = g["GDD15"].cumsum()
        hit = g[(g["roll5_Tmean"]>15.0) & (g["cumGDD15"]>=60.0)]
        emer = hit["Date"].iloc[0] if not hit.empty else pd.NaT
        rows.append({"Name":name,"Year":yr,"Emergence":emer})
    return pd.DataFrame(rows)

# ===== Stage tagging =====
def tag_cotton_stages(d: pd.DataFrame, emer_tbl: pd.DataFrame) -> pd.DataFrame:
    m = d.merge(emer_tbl, on=["Name","Year"], how="left")
    m["DAE"] = (m["Date"] - m["Emergence"]).dt.days
    # cotton-specific GDD for aggregation
    m["GDD_day"] = np.maximum(0.0, m["tmean_C"] - COTTON_GDD_BASE)

    def stage_from_dae(v):
        if pd.isna(v): return None
        for lab, lo, hi in COTTON_STAGES:
            if lo <= v <= hi: return lab
        return None

    m["Stage"] = m["DAE"].apply(stage_from_dae)
    return m

# ===== Streak helper (reuse if not in scope) =====
def longest_streak_bool(arr: pd.Series) -> int:
    m = 0; cur = 0
    for v in arr.astype(bool):
        if v: cur += 1; m = max(m, cur)
        else: cur = 0
    return m

# ===== Aggregate features by Stage =====
def cotton_stage_aggregate(tagged: pd.DataFrame) -> pd.DataFrame:
    g = tagged.dropna(subset=["Stage"]).copy().sort_values(["Name","Year","Stage","Date"])
    core = (g.groupby(["Name","Year","Stage"], as_index=False)
              .agg(
                  # thermal
                  GDD_sum=("GDD_day","sum"),
                  HeatDays=("HeatDay","sum"),
                  FrostDays=("FrostDay","sum"),
                  Tmean_C=("tmean_C","mean"),
                  Tmax_C=("tmax_C","mean"),
                  ThermalAmp=("ThermalAmp","mean"),
                  # water
                  PPT_mm=("ppt_mm","sum"),
                  ET0_sum=("ET0_proxy","sum"),
                  P_minus_ET=("P_minus_ET","sum"),
                  DryDays=("DryDay","sum"),
                  HeavyRainDays=("HeavyRain","sum"),
                  # VPD / humidity
                  VPDmax_kPa=("vpdmax_kPa","mean"),
                  VPDmean_kPa=("vpdmean_kPa","mean"),
                  VPD_hi_days=("VPD_hi_day","sum"),
                  VPD_load_sum=("VPD_load","sum"),
                  DewDep_C=("DewDep_C","mean"),
              ))
    grp = g.groupby(["Name","Year","Stage"], observed=True)
    streaks = (pd.DataFrame({
        "LongestDryStreak": grp["DryDay"].apply(longest_streak_bool),
        "LongestHotStreak": grp["HeatDay"].apply(longest_streak_bool),
    }).reset_index())
    out = core.merge(streaks, on=["Name","Year","Stage"], how="left")
    out["EffPrecRatio_idx"] = out["PPT_mm"] / out["ET0_sum"].replace(0, np.nan)
    return out

# ===== RUN =====
# Requires you already have: load_prism_all(DATA_DIR, PATTERN), build_daily(raw)
raw   = load_prism_all(DATA_DIR, PATTERN)
daily = build_daily(raw)

emer  = detect_emergence_cotton(daily)
cot_tagged = tag_cotton_stages(daily, emer)
cot_stage  = cotton_stage_aggregate(cot_tagged)

# save long
cot_tagged.to_parquet(OUTDIR/"cotton_daily_with_stages.parquet", index=False)
cot_stage.to_csv(OUTDIR/"cotton_stage_features.csv", index=False)

# wide per Name×Year
wide = (cot_stage.pivot_table(
    index=["Name","Year"], columns="Stage",
    values=["PPT_mm","ET0_sum","P_minus_ET","EffPrecRatio_idx",
            "GDD_sum","HeatDays","FrostDays","LongestDryStreak","LongestHotStreak",
            "Tmean_C","Tmax_C","ThermalAmp",
            "VPDmax_kPa","VPDmean_kPa","VPD_hi_days","VPD_load_sum","DewDep_C"],
    aggfunc="first").reset_index())
wide.columns = ["_".join(c).strip("_") for c in wide.columns.to_flat_index()]
wide.to_csv(OUTDIR/"cotton_stage_features_wide_plus.csv", index=False)

print("Cotton stage rows:", len(cot_stage), "| wide rows:", wide.shape)

# ===== Merge yields ≤2022 and prepare 2023 inference =====
yld = pd.read_csv(YIELD_CSV)

# keep upland cotton yields in lb/acre
is_cotton = yld["Commodity"].str.upper().str.contains("COTTON")
yld_cot = (yld.loc[is_cotton, ["Year","County","Commodity","YIELD_LBS_PER_ACRE"]]
              .rename(columns={"County":"Name","YIELD_LBS_PER_ACRE":"Yield"}))

cot_df = (yld_cot.merge(wide[wide["Year"]<=2022], on=["Name","Year"], how="inner")
                 .dropna()
                 .drop_duplicates(subset=["Name","Year"]))
cot_df.to_csv(OUTDIR/"cotton_model_table.csv", index=False)

cot_infer_2023 = (wide[wide["Year"]==2023]
                  .dropna()
                  .drop_duplicates(subset=["Name","Year"]))
cot_infer_2023.to_csv(OUTDIR/"cotton_inference_2023.csv", index=False)

print("Model table:", cot_df.shape, "| 2023 inference:", cot_infer_2023.shape)



Cotton stage rows: 25200 | wide rows: (4200, 104)
Model table: (1541, 106) | 2023 inference: (100, 104)


In [ ]:
# ===== Baseline model → 2023 predictions =====
FEATS = [c for c in cot_df.columns if c not in ["Name","Year","Commodity","Yield"]]

try:
    from xgboost import XGBRegressor
    model = XGBRegressor(
        n_estimators=600, max_depth=6, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0, random_state=42
    )
except Exception:
    from sklearn.ensemble import RandomForestRegressor
    model = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)

model.fit(cot_df[FEATS], cot_df["Yield"])
pred23 = model.predict(cot_infer_2023[FEATS])

predictions_cotton = pd.DataFrame({
    "Year": 2023,
    "Commodity": "COTTON",
    "Name": cot_infer_2023["Name"].values,
    "Value": pred23
})
predictions_cotton.to_csv(OUTDIR/"cotton_predictions_2023.csv", index=False)
print("Saved:", OUTDIR/"cotton_predictions_2023.csv")


In [103]:
from pathlib import Path
import pandas as pd

# ensure OUTDIR exists and show where we're writing
OUTDIR = Path("Kashish_results/corn"); OUTDIR.mkdir(parents=True, exist_ok=True)
print("Writing to:", OUTDIR.resolve())

corn_infer_2023 = (
    wide.loc[wide["Year"]==2023]
        .drop_duplicates(subset=["Name","Year"])
        .copy()
)

out_file = OUTDIR / "corn_inference_2023.csv"
corn_infer_2023.to_csv(out_file, index=False)
print("Saved:", out_file.resolve(), "| shape:", corn_infer_2023.shape)


Writing to: /home/ec2-user/SageMaker/PSI Hackathon/Kashish_results/corn_inference
Saved: /home/ec2-user/SageMaker/PSI Hackathon/Kashish_results/corn_inference/corn_inference_2023.csv | shape: (100, 104)


In [68]:
import pandas as pd
from pathlib import Path

OUTDIR = Path("Kashish_results"); OUTDIR.mkdir(exist_ok=True, parents=True)

# 1) Load yields and KEEP ONLY winter wheat BU/acre
WINTER_ITEM = "WHEAT, WINTER - YIELD, MEASURED IN BU / ACRE"
yld = pd.read_csv("crop_yield_1980-2022.csv")

w_winter = (yld.loc[yld["Data Item"].eq(WINTER_ITEM),
                    ["Year","County","Commodity","Data Item","YIELD_LBS_PER_ACRE"]]
              .rename(columns={"County":"Name","YIELD_LBS_PER_ACRE":"Yield"}))

print("Winter wheat rows:", len(w_winter), "years:", w_winter["Year"].min(), "-", w_winter["Year"].max())

# 2) Climate features (wide) and restrict to ≤2022
wide = pd.read_csv("/home/ec2-user/SageMaker/PSI Hackathon/Kashish_results/wheat/wheat_stage_features_wide_plus.csv")
wide_learn = wide[wide["Year"] <= 2022].copy()

# 3) Coverage diagnostics: which Name–Year in climate lack winter-wheat yield
clim_keys = wide_learn[["Name","Year"]].drop_duplicates()
yld_keys  = w_winter[["Name","Year"]].drop_duplicates()

missing_yield = (clim_keys.merge(yld_keys, on=["Name","Year"], how="left", indicator=True)
                         .query("_merge=='left_only'")
                         .drop(columns="_merge"))
missing_yield.to_csv(OUTDIR/"wheat_winter_missing_name_year.csv", index=False)
print("Climate-only (no winter yield) rows:", len(missing_yield))

# 4) Merge ONLY winter-wheat yields with climate features
wht_df = (w_winter.merge(wide_learn, on=["Name","Year"], how="inner")
                   .dropna()
                   .drop_duplicates(subset=["Name","Year"]))

# keep Data Item in the model table
wht_df.to_csv(OUTDIR/"wheat_model_table_winter_only.csv", index=False)
print("Model table (winter-only):", wht_df.shape)

# 5) Optional: show a few missing cases
print(missing_yield.sort_values(["Name","Year"]).head(20))


Winter wheat rows: 3313 years: 1980 - 2022
Climate-only (no winter yield) rows: 1064
Model table (winter-only): (2836, 107)
         Name  Year
64  ALEXANDER  2009
65  ALEXANDER  2010
68  ALEXANDER  2013
71  ALEXANDER  2016
72  ALEXANDER  2017
77  ALEXANDER  2022
80  ALLEGHANY  1986
81  ALLEGHANY  1987
82  ALLEGHANY  1988
83  ALLEGHANY  1989
84  ALLEGHANY  1990
85  ALLEGHANY  1991
86  ALLEGHANY  1992
87  ALLEGHANY  1993
88  ALLEGHANY  1994
89  ALLEGHANY  1995
90  ALLEGHANY  1996
94  ALLEGHANY  2000
95  ALLEGHANY  2001
96  ALLEGHANY  2002


In [69]:
import pandas as pd
from pathlib import Path

OUTDIR = Path("Kashish_results")

# load the full yield dataset
yld = pd.read_csv("crop_yield_1980-2022.csv")

# define items of interest
all_items = [
    "WHEAT - YIELD, MEASURED IN BU / ACRE",
    "WHEAT - YIELD, MEASURED IN BU / NET PLANTED ACRE",
    "WHEAT, WINTER - YIELD, MEASURED IN BU / ACRE",
    "WHEAT, WINTER - YIELD, MEASURED IN BU / NET PLANTED ACRE"
]

wheat_all = yld[yld["Data Item"].isin(all_items)].copy()
wheat_all = wheat_all.rename(columns={"County":"Name","YIELD_LBS_PER_ACRE":"Yield"})

# load the missing Name-Year list (from your previous step)
missing = pd.read_csv(OUTDIR/"wheat_winter_missing_name_year.csv")

# check which of the other items are available for those missing
alts = (wheat_all.merge(missing, on=["Name","Year"], how="inner")
                  .sort_values(["Name","Year","Data Item"]))

alts.to_csv(OUTDIR/"wheat_missing_winter_with_other_items.csv", index=False)

print("Alternative yields found for missing winter-wheat rows:", len(alts))
print(alts.groupby("Data Item")["Name"].count())
print(alts.head(20))


Alternative yields found for missing winter-wheat rows: 0
Series([], Name: Name, dtype: int64)
Empty DataFrame
Columns: [Unnamed: 0, Program, Year, Period, Week Ending, Geo Level, State, State ANSI, Ag District, Ag District Code, Name, County ANSI, Zip Code, Region, watershed_code, Watershed, Commodity, Data Item, Domain, Domain Category, Value, CV (%), Yield]
Index: []

[0 rows x 23 columns]


In [73]:
import pandas as pd
import numpy as np

# paths
model_fp = "/home/ec2-user/SageMaker/PSI Hackathon/Kashish_results/corn/corn_model_table.csv"     # has: Year, Name, Yield
usda_fp  = "/home/ec2-user/SageMaker/PSI Hackathon/crop_yield_1980-2022.csv"        # has: Year, County, Data Item, Value or YIELD_LBS_PER_ACRE

# load
model = pd.read_csv(model_fp)
usda  = pd.read_csv(usda_fp)

# normalize county keys
def norm_key(s):
    return (s.astype(str).str.upper()
            .str.replace(r"\s+COUNTY$", "", regex=True)
            .str.replace(r"\s+", " ", regex=True)
            .str.strip())

model["CountyKey"] = norm_key(model["Name"])
if "County" not in usda.columns:
    raise ValueError("USDA file must contain a 'County' column.")
usda["CountyKey"] = norm_key(usda["County"])

# --- select silage rows ---
mask_silage = usda["Data Item"].str.contains("CORN, SILAGE - YIELD, MEASURED IN TONS / ACRE", na=False)

sil = usda.loc[mask_silage, ["Year","CountyKey"]].copy()

# get numeric yield for silage (lbs/acre)
if "Value" in usda.columns:
    # Value likely in tons/acre -> convert to lbs/acre
    val = (usda.loc[mask_silage, "Value"]
              .astype(str).str.replace(",", "", regex=False)
              .str.extract(r"([-+]?\d*\.?\d+)")[0].astype(float))
    sil["Silage_Yield_LBS_PER_ACRE"] = val * 2000.0
elif "YIELD_LBS_PER_ACRE" in usda.columns:
    # already provided in lbs/acre
    val = (usda.loc[mask_silage, "YIELD_LBS_PER_ACRE"]
              .astype(str).str.replace(",", "", regex=False)
              .str.extract(r"([-+]?\d*\.?\d+)")[0].astype(float))
    sil["Silage_Yield_LBS_PER_ACRE"] = val
else:
    raise ValueError("USDA file needs either 'Value' or 'YIELD_LBS_PER_ACRE'.")

# de-duplicate if multiple entries per Year×County
sil = (sil.groupby(["Year","CountyKey"], as_index=False)
          .agg({"Silage_Yield_LBS_PER_ACRE":"mean"}))

# merge into model (keeps your existing grain Yield column)
out = model.merge(sil, how="left", on=["Year","CountyKey"]).drop(columns=["CountyKey"])

# report coverage
matched = out["Silage_Yield"].notna().sum()
print(f"Model rows: {len(out)}")
print(f"Silage yields matched: {matched}")
print(f"Missing silage yields: {len(out)-matched}")

# save
out.to_csv("Kashish_results/corn/corn_model_with_silage.csv", index=False)


Model rows: 3327
Silage yields matched: 823
Missing silage yields: 2504


In [76]:
# eda_all_crops.py
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

sns.set(style="whitegrid")

IN = {
    "CORN":     "/home/ec2-user/SageMaker/PSI Hackathon/Kashish_results/corn/corn_model_with_silage.csv",
    "SOYBEANS": "/home/ec2-user/SageMaker/PSI Hackathon/Kashish_results/soybean/soybean_model_table.csv",
    "PEANUTS":  "/home/ec2-user/SageMaker/PSI Hackathon/Kashish_results/peanuts/peanut_model_table.csv",
    "COTTON":   "/home/ec2-user/SageMaker/PSI Hackathon/Kashish_results/cotton/cotton_model_table.csv",
    "WHEAT":    "/home/ec2-user/SageMaker/PSI Hackathon/Kashish_results/wheat/wheat_model_table_winter_only.csv",  # adjust if different
}
OUTROOT = Path("Kashish_results/eda"); OUTROOT.mkdir(parents=True, exist_ok=True)
TARGET = "Yield"
DROP   = {"Name","Commodity"}  # dropped from features if present

def safe_num_df(df: pd.DataFrame) -> pd.DataFrame:
    num = df.select_dtypes(include=[np.number]).copy()
    # remove columns with zero variance
    nunique = num.nunique()
    keep = nunique[nunique > 1].index
    return num[keep]

def eda_one(crop: str, path: str):
    outdir = OUTROOT / crop; outdir.mkdir(parents=True, exist_ok=True)
    df = pd.read_csv(path)
    print(f"[{crop}] rows={len(df)} cols={df.shape[1]}")

    # ---------- 1) Yield over time + distribution ----------
    plt.figure(figsize=(11,4))
    
    sns.lineplot(data=df, x="Year", y=TARGET, estimator="mean", errorbar=None)

    plt.title(f"{crop}: Mean Yield over Time"); plt.ylabel("Yield (lbs/acre)")
    plt.tight_layout(); plt.savefig(outdir/"yield_over_time.png", dpi=160); plt.close()

    plt.figure(figsize=(7,4))
    sns.histplot(df[TARGET], bins=40, kde=True)
    plt.title(f"{crop}: Yield Distribution")
    plt.tight_layout(); plt.savefig(outdir/"yield_hist.png", dpi=160); plt.close()

    # ---------- 2) Correlations ----------
    num = safe_num_df(df)
    if TARGET not in num.columns:
        num[TARGET] = df[TARGET].values
    corr = num.corr()

    # save full correlation matrix
    corr.to_csv(outdir/"corr_matrix.csv")

    # heatmap vs target
    ct = corr[[TARGET]].sort_values(by=TARGET, ascending=False)
    plt.figure(figsize=(6, min(0.35*len(ct), 16)))
    sns.heatmap(ct, annot=True, fmt=".2f", cmap="coolwarm", center=0, cbar=False)
    plt.title(f"{crop}: Correlation with Yield")
    plt.tight_layout(); plt.savefig(outdir/"corr_with_yield.png", dpi=160); plt.close()

    # ---------- 3) Scatterplots of top correlated ----------
    top_feats = (ct.index.drop(TARGET)
                   .to_series().reindex(ct.index.drop(TARGET))
    )
    top5 = (corr[TARGET].drop(TARGET).abs().sort_values(ascending=False).head(5).index.tolist())
    for feat in top5:
        plt.figure(figsize=(5.5,4))
        sns.scatterplot(data=df, x=feat, y=TARGET, alpha=0.5)
        plt.title(f"{crop}: Yield vs {feat}")
        plt.tight_layout(); plt.savefig(outdir/f"scatter_{feat}.png", dpi=160); plt.close()

    pd.Series(top5, name="Top5_by_abs_corr").to_csv(outdir/"top5_corr_features.csv", index=False)

    # ---------- 4) Feature importance (RF) ----------
    X = df.drop(columns=[c for c in [TARGET, *DROP] if c in df.columns], errors="ignore")
    # keep only numeric columns
    X = X.select_dtypes(include=[np.number]).copy()
    # remove zero-variance
    nunique = X.nunique()
    X = X.loc[:, nunique[nunique>1].index]
    y = df[TARGET].values

    if X.shape[1] >= 2:
        Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)
        rf = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
        rf.fit(Xtr, ytr)
        imp = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
        imp.to_csv(outdir/"rf_feature_importances.csv")
        top15 = imp.head(15)

        plt.figure(figsize=(9, max(4, 0.35*len(top15))))
        sns.barplot(x=top15.values, y=top15.index)
        plt.title(f"{crop}: Top RF Feature Importances")
        plt.xlabel("Importance"); plt.ylabel("")
        plt.tight_layout(); plt.savefig(outdir/"rf_feature_importances_top15.png", dpi=160); plt.close()
    else:
        print(f"[{crop}] Skipped RF importance (insufficient numeric features).")

for crop, path in IN.items():
    try:
        eda_one(crop, path)
    except Exception as e:
        print(f"[{crop}] ERROR: {e}")

print("EDA complete. See:", OUTROOT.resolve())


[CORN] rows=3327 cols=158
[SOYBEANS] rows=3096 cols=123
[PEANUTS] rows=938 cols=106
[COTTON] rows=1541 cols=106
[WHEAT] ERROR: [Errno 2] No such file or directory: '/home/ec2-user/SageMaker/PSI Hackathon/Kashish_results/wheat/wheat_model_table_winter_only.csv'
EDA complete. See: /home/ec2-user/SageMaker/PSI Hackathon/Kashish_results/eda


In [90]:
from pathlib import Path
import matplotlib.pyplot as plt

# set these if not already
crop = "CORN"
best_col = "VPDmean_kPa_R5"

# byyr must already exist with Year, Yield_z, Stress_z
bad = byyr[(byyr["Yield_z"] <= -0.5) & (byyr["Stress_z"] >= 0.5)]

outdir = Path(f"Kashish_results/stress_analysis_v2/{crop}")
outdir.mkdir(parents=True, exist_ok=True)

plt.figure(figsize=(10,5))
plt.plot(byyr["Year"], byyr["Yield_z"], color="blue", label="Yield (z)")
plt.plot(byyr["Year"], byyr["Stress_z"], color="red", linestyle="--", label=f"{best_col} (z)")
plt.axhline(0, color="gray", linewidth=0.8)
for yr in bad["Year"]:
    plt.axvspan(yr-0.5, yr+0.5, color="orange", alpha=0.2)

plt.title(f"{crop}: Yield vs {best_col} (z-scores)")
plt.xlabel("Year"); plt.ylabel("z-score"); plt.legend()
plt.tight_layout()
plt.savefig(outdir / f"yearly_zscores_better_{best_col}.png", dpi=200)
plt.close()

bad[["Year","Yield_z","Stress_z"]].to_csv(outdir / f"bad_years_{best_col}.csv", index=False)


In [100]:
# stress_finder_all_crops.py
import numpy as np, pandas as pd, re
from pathlib import Path
import matplotlib.pyplot as plt, seaborn as sns
sns.set(style="whitegrid")

# ===== CONFIG =====
IN = {
    "CORN":     "final_dataset/Corn_normalized.csv",
  #  "SOYBEANS": "final_dataset/Soybean_normalized.csv",
   # "PEANUTS":  "final_dataset/Peanut_normalized.csv",
    #"COTTON":   "final_dataset/Cotton_normalized.csv",
    #"WHEAT":    "final_dataset/Wheat_normalized.csv",
}
OUTROOT = Path("Kashish_results/stress_analysis_v3"); OUTROOT.mkdir(parents=True, exist_ok=True)
TARGET = "Yield"

STRESS_PREFIXES = {
    # include only stress-like metrics
    "COMMON": ["HeatDays","LongestHotStreak","DryDays","LongestDryStreak",
               "P_minus_ET","EffPrecRatio_idx",
               "VPD_hi_days","VPD_load_sum","VPDmax_kPa","VPDmean_kPa",
               "FrostDays"]
}

STAGE_TOKENS = {
    "CORN":     ["VE","V6","VT","R1","R2","R3","R4","R5","R6"],
    "SOYBEANS": ["VE","V2_V3","V4_V6","R1_R2","R3_R4","R5_R6","R7_R8"],
    "PEANUTS":  ["VE","VEG","PEG","POD_SET","SEED_FILL","MATURITY"],   # adjust to your columns if needed
    "COTTON":   ["VE","VEG","SQUARING","FLOWERING","BOLL_DEV","OPEN_MAT"],
    "WHEAT":    ["TILLER","STEM_ELONG","BOOT","HEAD_FLOWER","GRAIN_FILL","RIPEN"],
}

MIN_NEG = -0.10
TOP_K   = 5

# ===== HELPERS =====
def zscore(s: pd.Series):
    v = (s - s.mean())/s.std(ddof=0)
    return v.replace([np.inf,-np.inf], np.nan)

def split_by_stage_suffix(col: str, crop: str):
    """Robust split: if col endswith '_<stage>' for any known stage token, use that.
       Else fallback to regex last-underscore split."""
    for st in STAGE_TOKENS.get(crop, []):
        suf = "_"+st
        if col.endswith(suf):
            return col[:-len(suf)], st
    m = re.match(r"^(.*)_([A-Z0-9]+(?:_[A-Z0-9]+)?)$", col)
    return m.groups() if m else (None, None)

def stage_feature_index(df: pd.DataFrame, crop: str, target: str):
    rows=[]
    skip = {target,"Year","Name","COUNTY","Commodity","Data Item","Unnamed: 0"}
    prefixes = STRESS_PREFIXES["COMMON"]
    for c in df.columns:
        if c in skip: 
            continue
        metric, stage = split_by_stage_suffix(c, crop)
        if not metric or not stage:
            continue
        if any(metric.startswith(p) for p in prefixes):
            rows.append((c, metric, stage))
    return pd.DataFrame(rows, columns=["col","metric","stage"])

def nice_stage_order(stages, crop):
    pref = STAGE_TOKENS.get(crop, [])
    seen = [s for s in pref if s in stages]
    rest = [s for s in stages if s not in pref]
    return seen + rest

# ===== PLOTTING =====
def heatmap_corr(corr_mat: pd.DataFrame, title: str, out_png: Path):
    plt.figure(figsize=(1.2*corr_mat.shape[1]+2, 0.55*corr_mat.shape[0]+2))
    sns.heatmap(corr_mat, annot=True, fmt=".2f", cmap="coolwarm", center=0,
                linewidths=.5, linecolor="white", cbar_kws={"shrink":0.8})
    plt.title(title); plt.tight_layout(); plt.savefig(out_png, dpi=200); plt.close()

def scatter_with_fit(df, xcol, ycol, title, out_png):
    plt.figure(figsize=(6.4,4.6))
    sns.scatterplot(data=df, x=xcol, y=ycol, alpha=0.35, edgecolor=None)
    sns.regplot(data=df, x=xcol, y=ycol, scatter=False, color="black")
    plt.title(title); plt.tight_layout(); plt.savefig(out_png, dpi=200); plt.close()

def dual_z_trend(byyr, crop, best_col, out_png):
    plt.figure(figsize=(10,5))
    plt.plot(byyr["Year"], byyr["Yield_z"],  color="blue", label="Yield (z)")
    plt.plot(byyr["Year"], byyr["Stress_z"], color="red",  linestyle="--", label=f"{best_col} (z)")
    plt.axhline(0, color="gray", linewidth=0.8)
    bad = byyr[(byyr["Yield_z"]<=-0.5) & (byyr["Stress_z"]>=0.5)]
    for yr in bad["Year"]:
        plt.axvspan(yr-0.5, yr+0.5, color="orange", alpha=0.2)
    plt.title(f"{crop}: Yield vs {best_col} (z-scores)")
    plt.xlabel("Year"); plt.ylabel("z-score"); plt.legend()
    plt.tight_layout(); plt.savefig(out_png, dpi=200); plt.close()
    return bad

# ===== CORE =====
def analyze_crop(crop: str, path: str):
    outdir = OUTROOT / crop; outdir.mkdir(parents=True, exist_ok=True)
    df = pd.read_csv(path)

    # IDs present?
    id_cols = [c for c in ["Name","COUNTY"] if c in df.columns]
    key_cols = ["Year", TARGET] + id_cols

    # index stress-stage columns using robust splitter
    sf = stage_feature_index(df, crop, TARGET)
    if sf.empty:
        print(f"[{crop}] No stress-stage columns detected. Check column names.")
        return

    # diagnostics to catch “only VE”
    stage_counts = sf["stage"].value_counts().sort_index()
    stage_counts.to_csv(outdir/"_debug_stage_counts.csv")
    print(f"[{crop}] Stages found:", ", ".join(stage_counts.index.tolist()))

    use_cols = key_cols + sf["col"].tolist()
    df = df[use_cols].dropna(subset=[TARGET]).copy()

    # 1) correlation heatmap
    stages = nice_stage_order(sorted(sf["stage"].unique()), crop)
    metrics = sorted(sf["metric"].unique())
    corr_mat = pd.DataFrame(index=metrics, columns=stages, dtype=float)
    for met in metrics:
        for st in stages:
            col = f"{met}_{st}"
            corr_mat.loc[met, st] = df[[TARGET, col]].corr().iloc[0,1] if col in df.columns else np.nan
    corr_mat.to_csv(outdir/"corr_stress_stage_vs_yield.csv")
    heatmap_corr(corr_mat, f"{crop}: Correlation (Stress×Stage) vs Yield", outdir/"corr_heatmap.png")

    # 2) top harmful (most negative)
    cm_long = corr_mat.stack(dropna=True).rename("corr").reset_index()
    cm_long.columns = ["metric","stage","corr"]
    neg = cm_long[cm_long["corr"] < 0]
    if neg.empty:
        top = cm_long.sort_values("corr").head(1)
    else:
        base = neg[neg["corr"] <= MIN_NEG] if (neg["corr"] <= MIN_NEG).any() else neg
        top = base.sort_values("corr").head(1)

    topk = (neg.sort_values("corr").head(TOP_K) if not neg.empty
            else cm_long.sort_values("corr").head(TOP_K))
    topk.to_csv(outdir/"top5_harmful_stress.csv", index=False)

    
    metric, stage, corr_val = top.iloc[0]["metric"], top.iloc[0]["stage"], float(top.iloc[0]["corr"])
    best_col = f"{metric}_{stage}"
    pd.Series({"metric":metric,"stage":stage,"corr":corr_val}).to_csv(outdir/"top_harmful_stress.csv")

    # 3) yearly stress vs yield
    byyr = (df.groupby("Year", as_index=False)
              .agg(Yield_mean=(TARGET,"mean"),
                   Stress_mean=(best_col,"mean")))
    byyr["Yield_z"]  = zscore(byyr["Yield_mean"])
    byyr["Stress_z"] = zscore(byyr["Stress_mean"])
    byyr.to_csv(outdir/"year_trends_top_stress.csv", index=False)

    # improved plot + bad years
    bad = dual_z_trend(byyr, crop, best_col, outdir/"yearly_zscores_better.png")
    bad.to_csv(outdir/"years_high_stress_low_yield.csv", index=False)

    # 4) scatter
    scatter_with_fit(df, best_col, TARGET,
                     f"{crop}: Yield vs {best_col} (r={corr_val:.2f})",
                     outdir/f"scatter_{best_col}.png")

    # optional per-county correlation
    if id_cols:
        def _corr(g):
            if g[TARGET].nunique()<3 or g[best_col].nunique()<3: return np.nan
            return g[[TARGET, best_col]].corr().iloc[0,1]
        by_cty = (df.groupby(id_cols, as_index=False).apply(_corr)
                    .rename(columns={None:"corr"}))
        by_cty.to_csv(outdir/"county_corr_with_top_stress.csv", index=False)

    print(f"[{crop}] Top harmful: {best_col} r={corr_val:.3f}  |  stages={stage_counts.to_dict()}  |  → {outdir}")

# ===== RUN =====
for crop, path in IN.items():
    try:
        analyze_crop(crop, path)
    except Exception as e:
        print(f"[{crop}] ERROR: {e}")

print("Done →", OUTROOT.resolve())


[CORN] Stages found: R1, R2, R3, R4, R5, R6, V6, VE, VT


/tmp/ipykernel_16293/2466995035.py:133: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  cm_long = corr_mat.stack(dropna=True).rename("corr").reset_index()


[CORN] Top harmful: VPDmean_kPa_R5 r=-0.311  |  stages={'R1': 10, 'R2': 10, 'R3': 10, 'R4': 10, 'R5': 10, 'R6': 10, 'V6': 10, 'VE': 10, 'VT': 10}  |  → Kashish_results/stress_analysis_v3/CORN
Done → /home/ec2-user/SageMaker/PSI Hackathon/Kashish_results/stress_analysis_v3


/tmp/ipykernel_16293/2466995035.py:173: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  by_cty = (df.groupby(id_cols, as_index=False).apply(_corr)


In [108]:
# corn_pipeline.py

import pandas as pd, numpy as np
from pathlib import Path

# ========= CONFIG =========
DATA_DIR = Path("Daily_for_transform/Daily Climate Data")
PATTERN  = "PRISM_ppt_tmin_tmean_tmax_tdmean_vpdmin_vpdmax_stable_800m_*.csv"
OUTDIR   = Path("Kashish_results/corn"); OUTDIR.mkdir(parents=True, exist_ok=True)
YIELD_CSV = "crop_yield_1980-2022.csv"   # change if different
TARGET = "Yield"

# ========= ASSUMED AVAILABLE (from your session) =========
# - load_prism_all(DATA_DIR, PATTERN)
# - build_daily(raw)  -> includes ppt_mm, t*, VPD*, ET0_proxy, P_minus_ET, HeatDay, DryDay, FrostDay, ThermalAmp, DewDep_C
# - detect_emergence(daily)
# - longest_streak_bool(series_of_bool) -> max consecutive True

# ========= Corn phenology (DAE) =========
CORN_STAGES = [
    ("VE",   0,   10),
    ("V6",  25,   30),
    ("VT",  55,   59),
    ("R1",  60,   70),
    ("R2",  71,   80),
    ("R3",  81,   90),
    ("R4",  91,  105),
    ("R5", 106,  125),
    ("R6", 126,  160),
]

def tag_corn_stages(d: pd.DataFrame, emer_tbl: pd.DataFrame) -> pd.DataFrame:
    m = d.merge(emer_tbl, on=["Name","Year"], how="left")
    m["DAE"] = (m["Date"] - m["Emergence"]).dt.days
    def stage_from_dae(v):
        if pd.isna(v): return None
        for lab, lo, hi in CORN_STAGES:
            if lo <= v <= hi: return lab
        return None
    m["Stage"] = m["DAE"].apply(stage_from_dae)
    return m

def corn_stage_aggregate(tagged: pd.DataFrame) -> pd.DataFrame:
    g = tagged.dropna(subset=["Stage"]).copy()
    g = g.sort_values(["Name","Year","Stage","Date"])

    core = (g.groupby(["Name","Year","Stage"], as_index=False)
              .agg(
                  # thermal
                  GDD_sum=("GDD_day","sum"),
                  HeatDays=("HeatDay","sum"),
                  FrostDays=("FrostDay","sum"),
                  Tmean_C=("tmean_C","mean"),
                  Tmax_C=("tmax_C","mean"),
                  ThermalAmp=("ThermalAmp","mean"),
                  # water
                  PPT_mm=("ppt_mm","sum"),
                  ET0_sum=("ET0_proxy","sum"),
                  P_minus_ET=("P_minus_ET","sum"),
                  DryDays=("DryDay","sum"),
                  HeavyRainDays=("HeavyRain","sum"),
                  # VPD / humidity
                  VPDmax_kPa=("vpdmax_kPa","mean"),
                  VPDmean_kPa=("vpdmean_kPa","mean"),
                  VPD_hi_days=("VPD_hi_day","sum"),
                  VPD_load_sum=("VPD_load","sum"),
                  DewDep_C=("DewDep_C","mean"),
              ))

    grp = g.groupby(["Name","Year","Stage"], observed=True)
    streaks = (pd.DataFrame({
        "LongestDryStreak": grp["DryDay"].apply(longest_streak_bool),
        "LongestHotStreak": grp["HeatDay"].apply(longest_streak_bool),
    }).reset_index())

    out = core.merge(streaks, on=["Name","Year","Stage"], how="left")
    out["EffPrecRatio_idx"] = out["PPT_mm"] / out["ET0_sum"].replace(0, np.nan)
    return out

# ========= RUN: features =========
raw   = load_prism_all(DATA_DIR, PATTERN)
daily = build_daily(raw)
emer  = detect_emergence(daily)

corn_tagged = tag_corn_stages(daily, emer)
corn_stage  = corn_stage_aggregate(corn_tagged)

corn_tagged.to_parquet(OUTDIR/"corn_daily_with_stages.parquet", index=False)
corn_stage.to_csv(OUTDIR/"corn_stage_features_long.csv", index=False)

# wide table
wide = (corn_stage.pivot_table(
            index=["Name","Year"], columns="Stage",
            values=["PPT_mm","ET0_sum","P_minus_ET","EffPrecRatio_idx",
                    "GDD_sum","HeatDays","FrostDays","LongestDryStreak","LongestHotStreak",
                    "Tmean_C","Tmax_C","ThermalAmp",
                    "VPDmax_kPa","VPDmean_kPa","VPD_hi_days","VPD_load_sum","DewDep_C"],
            aggfunc="first")
        .reset_index())
wide.columns = ["*".join(c).strip("*") for c in wide.columns.to_flat_index()]
wide.to_csv(OUTDIR/"corn_stage_features_wide.csv", index=False)

# ========= MERGE YIELD ≤2022 to build model table =========
yld = pd.read_csv(YIELD_CSV)

# normalize county name column and yield
def norm_county_col(df):
    for c in ["County","COUNTY","Name"]:
        if c in df.columns:
            df = df.rename(columns={c:"Name"})
            break
    df["Name"] = df["Name"].astype(str).str.strip()
    return df

yld = norm_county_col(yld)
# corn grain yield; use YIELD_LBS_PER_ACRE (already present in your file)
is_corn = yld["Commodity"].str.upper().str.contains("^CORN$")
grain   = yld["Data Item"].str.upper().str.contains("GRAIN - YIELD")
yld_corn = (yld.loc[is_corn & grain, ["Year","Name","Commodity","YIELD_LBS_PER_ACRE"]]
              .rename(columns={"YIELD_LBS_PER_ACRE": TARGET}))
yld_corn[TARGET] = pd.to_numeric(yld_corn[TARGET], errors="coerce")

wide2 = norm_county_col(wide)
model_tbl = (yld_corn.merge(wide2[wide2["Year"]<=2022], on=["Name","Year"], how="inner")
                     .dropna(subset=[TARGET]))
model_tbl.to_csv(OUTDIR/"corn_model_table1.csv", index=False)

# ========= INFERENCE SET FOR 2023 =========
infer_2023 = wide2[wide2["Year"]==2023].copy()
# keep IDs + features only
id_cols = ["Name","Year"]
feat_cols = [c for c in infer_2023.columns if c not in id_cols]
infer_2023 = infer_2023[id_cols + feat_cols]
infer_2023.to_csv(OUTDIR/"corn_inference_20231.csv", index=False)

print("Corn stage rows:", len(corn_stage), "| wide rows:", wide.shape)
print("Model table:", model_tbl.shape, "→", OUTDIR/"corn_model_table1.csv")
print("2023 inference:", infer_2023.shape, "→", OUTDIR/"corn_inference_20231.csv")


Corn stage rows: 37800 | wide rows: (4200, 155)
Model table: (3547, 157) → Kashish_results/corn/corn_model_table1.csv
2023 inference: (100, 155) → Kashish_results/corn/corn_inference_20231.csv


In [109]:
import pandas as pd
from pathlib import Path

src = Path("Kashish_results/corn/corn_inference_20231.csv")
df = pd.read_csv(src)

# replace '*' with '_' in all column names
df.columns = [c.replace("*", "_") for c in df.columns]

# save (new file to be safe)
dst = src.with_name(src.stem + "_colsfixed.csv")
df.to_csv(dst, index=False)
print("Wrote:", dst)


Wrote: Kashish_results/corn/corn_inference_20231_colsfixed.csv
